In [12]:
import gspread
import pandas as pd
from fractions import Fraction
import re

# Pastikan permision access di menu "Share" di rubah ke anyone with the link terlebih dahulu, agar bisa mengunakan cara OAuth
# Jika Ingin lebih dibatasi menggunakan cara google service account lebih disarankan

gc = gspread.oauth(
    credentials_filename= r'..\..\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1k-70BJNRm01RE2S7gBXuOfqxofk1gYz4g6lPbLUCklw'
SHEET_NAME = 'DB'


In [13]:
# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID)

# Get all sheet names
sheet_names = [sheet.title for sheet in spreadsheet_source.worksheets()]
sheet_names

['DB',
 'DV',
 'List Events Transition',
 'Sheet24',
 'Template',
 'Summary Report',
 'Past Attedants',
 'PAST EVENTS - Summary Report',
 'HOME ALONE IN CONCERT 5 Dec',
 'ASCOT FESTIVE DAY AT THE RACES 20 Dec',
 'CHARITY GALA 11 Dec',
 'A FEAST OF FITNESS IN GOA 10-14 Dec',
 'AYURVEDIC CURATIVES 6-9 Dec',
 '12 DAYS OF KARMA BAVARIA',
 '12 DAYS OF KARMA KK',
 '12 DAYS OF KARMA KLOM',
 '12 DAYS OF KARMA KSH',
 'ANCIENT BREWS OF THE AEGEAN 23-26 Nov',
 'A VIP EXPERIENCE WITH RUGBY LEGENDS! 9 Nov',
 'URBAN POLO: HIGH SOCIETY 16 Nov',
 'SPRITZ INTO SUMMER 17 Nov',
 'MELBOURNE MEMBER MIXER! 3 Nov',
 'HEART & SOUL OF OKTOBERFEST 24-27 Sep',
 'TACKLE, TRAIN & TOAST 27-30 Sep',
 'AWAKEN YOUR RADIANCE WITH BALIS GLOW GURU 4-7 Oct',
 'CARCASSONNE GASTRONOMY 11-13 Oct',
 'LITERATURE LOVERS WEEKEND 11-13 Oct',
 'DUSSEHRA: HIMACHALI RITUALS 11-14 Oct',
 'ANDALUSIA FITNESS & FLOW WITH JO 14-18 Oct',
 'HIDE & FEAST:CHEF JOES EPICUREAN 18-19 Oct',
 'SURREALIST SOJOURN IN SOUTHERN SPAIN 24-28 Oct',
 'SE

In [7]:
# get data gsheet

spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()

# print(rows[:5])

# print('==============================')
# df = pd.DataFrame(rows, dtype="string")
df = pd.DataFrame(rows)
# df['Mobile Number'] = df['Mobile Number'].apply(Fraction)
df

,No,Servicing Office,Membership Type,Member No,First Name,Last Name,Email,Tel/Phone,Status,Staff Name,...,Event Type,Event Status,Event Name,Event Sheet Link,Event Name Detail,Event Link,Available Spots,Quota Event,Point Used,Paid (Usd)
0,1,KCI,Karma Club,84079,Steve,Dean,steviedeanie@gmail.com,447740593820,Confirmed,Lyddea,...,Adventure,Cancelled,CHELTENHAM FEST 14 MAR,https://docs.google.com/spreadsheets/d/15bYOR2...,,https://edm.karmadestinations.com/karmagroup/2...,,,,
1,2,KCI,Karma Club,84079,Steve,Dean,steviedeanie@gmail.com,447740593820,Confirmed,Lyddea,...,Adventure,Cancelled,CHELTENHAM FEST 14 MAR,https://docs.google.com/spreadsheets/d/15bYOR2...,,https://edm.karmadestinations.com/karmagroup/2...,,,,
2,3,KCB,Karma Royal Residences,964127,Rupert,Banks,rupert.banks@btinternet.com,447768310900,Pending,Made,...,Adventure,Cancelled,CHELTENHAM FEST 14 MAR,https://docs.google.com/spreadsheets/d/15bYOR2...,,https://edm.karmadestinations.com/karmagroup/2...,,,,
3,4,KCB,Karma Royal Residences,932633,Gary,Chapple,garstcha@yahoo.com,7790886878,Pending,Rani,...,Adventure,Cancelled,CHELTENHAM FEST 14 MAR,https://docs.google.com/spreadsheets/d/15bYOR2...,,https://edm.karmadestinations.com/karmagroup/2...,,,,
4,5,KCB,Karma Royal Residences,947243,Adrian,Ralph,stanandann@aol.com,447923041816,Pending,Made,...,Adventure,Cancelled,CHELTENHAM FEST 14 MAR,https://docs.google.com/spreadsheets/d/15bYOR2...,,https://edm.karmadestinations.com/karmagroup/2...,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6637,25,,,,,,,,,,...,,Currently Working On,CHRISTMAS IN THE COTSWOLDS 5-8 DEC,https://docs.google.com/spreadsheets/d/1AfxaRO...,,https://karmaclub.karmagroup.com/karma-curated...,4,,,
6638,26,,,,,,,,,,...,,Currently Working On,CHRISTMAS IN THE COTSWOLDS 5-8 DEC,https://docs.google.com/spreadsheets/d/1AfxaRO...,,https://karmaclub.karmagroup.com/karma-curated...,4,,,
6639,1,KCI,,1009868,Aashka,Shukla,ashi131092@gmail.com,919328814939,,,...,,Currently Working On,FREDDIE’S HOMETOWN MUSICAL 8 -11 DEC,https://docs.google.com/spreadsheets/d/1AfxaRO...,,https://karmaclub.karmagroup.com/karma-curated...,10,,,
6640,2,KCI,Karma Club,56045,Subhash,Molwane,sdmolwane@yahoo.co.in,918369192252,,Virginia Rodrigues,...,,Currently Working On,FREDDIE’S HOMETOWN MUSICAL 8 -11 DEC,https://docs.google.com/spreadsheets/d/1AfxaRO...,,https://karmaclub.karmagroup.com/karma-curated...,10,,,


In [8]:
new_df = df.loc[(df['Event Name Detail'] != "") & 
                (df['Event Date From'] != "") & 
                (df['Event Date To'] != "")].copy()

In [9]:
# Convert 'Event Date From' and 'Event Date To' to datetime
new_df['Event Date From'] = pd.to_datetime(new_df['Event Date From'], errors='coerce')
new_df['Event Date To'] = pd.to_datetime(new_df['Event Date To'], errors='coerce')

# Create a new column for event name and feedback
def create_event_feedback(row):
    # Format the dates to "07 Apr"
    date_from = row['Event Date From'].strftime('%d %b')
    date_to = row['Event Date To'].strftime('%d %b')

    # Extract the year from 'Event Date To'
    year = row['Event Date To'].year
    
    if date_from == date_to:
        return f"{row['Event Name Detail']} ({date_from} {year})"
    else:
        return f"{row['Event Name Detail']} ({date_from} to {date_to} {year})"

new_df['Event Name Guest Feedback'] = new_df.apply(create_event_feedback, axis=1)
new_df

,No,Servicing Office,Membership Type,Member No,First Name,Last Name,Email,Tel/Phone,Status,Staff Name,...,Event Status,Event Name,Event Sheet Link,Event Name Detail,Event Link,Available Spots,Quota Event,Point Used,Paid (Usd),Event Name Guest Feedback
740,1,KCB,Karma Royal Residences,968973,Trevor,Heaslip,trevorheaslip2@gmail.com,61419270667,Confirmed,Sri,...,Sold Out,HOME ALONE IN CONCERT 5 DEC,https://docs.google.com/spreadsheets/d/1k-70BJ...,Member Mixer: Home Alone in Concert,https://karmaclub.karmagroup.com/karma-curated...,"Max 8 KC members, 6 HOK members","Max 8 KC members, 6 HOK members",,,Member Mixer: Home Alone in Concert (05 Dec 2024)
741,1,KCB,Karma Royal Residences,968973,Trevor,Heaslip,trevorheaslip2@gmail.com,61419270667,Confirmed,Sri,...,Sold Out,HOME ALONE IN CONCERT 5 DEC,https://docs.google.com/spreadsheets/d/1k-70BJ...,Member Mixer: Home Alone in Concert,https://karmaclub.karmagroup.com/karma-curated...,"Max 8 KC members, 6 HOK members","Max 8 KC members, 6 HOK members",,,Member Mixer: Home Alone in Concert (05 Dec 2024)
742,2,KCB,Karma Royal Residences,968973,Jacqueline,Heaslip,heaslip94@gmail.com,61433069319,Not Interested,Sri,...,Sold Out,HOME ALONE IN CONCERT 5 DEC,https://docs.google.com/spreadsheets/d/1k-70BJ...,Member Mixer: Home Alone in Concert,https://karmaclub.karmagroup.com/karma-curated...,"Max 8 KC members, 6 HOK members","Max 8 KC members, 6 HOK members",,,Member Mixer: Home Alone in Concert (05 Dec 2024)
743,3,KCB,Karma Club Discovery,1131454,Rodney,Butcher,rod1956@bigpond.com,61425718260,,Sri,...,Sold Out,HOME ALONE IN CONCERT 5 DEC,https://docs.google.com/spreadsheets/d/1k-70BJ...,Member Mixer: Home Alone in Concert,https://karmaclub.karmagroup.com/karma-curated...,"Max 8 KC members, 6 HOK members","Max 8 KC members, 6 HOK members",,,Member Mixer: Home Alone in Concert (05 Dec 2024)
744,4,KCB,Karma Club Discovery,1131454,Shelley,Butcher,rodbutcher1@bigpond.com,61425718260,Not Interested,Sri,...,Sold Out,HOME ALONE IN CONCERT 5 DEC,https://docs.google.com/spreadsheets/d/1k-70BJ...,Member Mixer: Home Alone in Concert,https://karmaclub.karmagroup.com/karma-curated...,"Max 8 KC members, 6 HOK members","Max 8 KC members, 6 HOK members",,,Member Mixer: Home Alone in Concert (05 Dec 2024)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5609,21,,,,,,,,,,...,Currently Working On,KOKAN FLAVOURS 7-10 APR,https://docs.google.com/spreadsheets/d/1AfxaRO...,Konkan Flavours: A Royal Culinary Journey,https://karmaclub.karmagroup.com/karma-curated...,7,,,,Konkan Flavours: A Royal Culinary Journey (07 ...
5610,22,,,,,,,,,,...,Currently Working On,KOKAN FLAVOURS 7-10 APR,https://docs.google.com/spreadsheets/d/1AfxaRO...,Konkan Flavours: A Royal Culinary Journey,https://karmaclub.karmagroup.com/karma-curated...,7,,,,Konkan Flavours: A Royal Culinary Journey (07 ...
5611,23,,,,,,,,,,...,Currently Working On,KOKAN FLAVOURS 7-10 APR,https://docs.google.com/spreadsheets/d/1AfxaRO...,Konkan Flavours: A Royal Culinary Journey,https://karmaclub.karmagroup.com/karma-curated...,7,,,,Konkan Flavours: A Royal Culinary Journey (07 ...
5612,24,,,,,,,,,,...,Currently Working On,KOKAN FLAVOURS 7-10 APR,https://docs.google.com/spreadsheets/d/1AfxaRO...,Konkan Flavours: A Royal Culinary Journey,https://karmaclub.karmagroup.com/karma-curated...,7,,,,Konkan Flavours: A Royal Culinary Journey (07 ...


In [10]:
new_df.loc[new_df['Event Name Detail'].str.contains('Himalayan Skiing & Winter Carnival Adventure', regex=True, flags=re.I)]

,No,Servicing Office,Membership Type,Member No,First Name,Last Name,Email,Tel/Phone,Status,Staff Name,...,Event Status,Event Name,Event Sheet Link,Event Name Detail,Event Link,Available Spots,Quota Event,Point Used,Paid (Usd),Event Name Guest Feedback
4749,1,KCI,Karma Club,69057,Baridalyne,Nongkynrih,baridalyneh@gmail.com,919811699865,Not Interested,Chrysanthi Fernandes,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4750,2,KCI,Karma Club,1085820,Preethi U,A,preethi.upadya@gmail.com,919886645025,Not Interested,Finosha,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4751,3,KCI,Karma Club,56816,Srinath,Subrahmanyan,srinath@umich.edu,919884112351,Pending,Chrysanthi Fernandes,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4752,4,KCI,Karma Club,1076004,Madhumita,Halder,rajat.dhariwal@gmail.com,919701075588,Interested,Finosha,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4753,5,KCB,Karma Royal Residences,25197,Adelheid,Johansson,peter.hultberg55@gmail.com,46707335856,Confirmed,Sri,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4754,5,KCB,Karma Royal Residences,25197,Adelheid,Johansson,peter.hultberg55@gmail.com,46707335856,Confirmed,Sri,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4755,6,KCI,,64194,Jay T.,Dalaya,jaydalaya@gmail.com,919920425180,Not Interested,,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4756,7,KCB,,1184127,Mervyn,Cussen,mervyn.cussen@gmail.com,61417303125,,Sri,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4757,8,KCI,Karma Club,131459,Uday V,Gada,rajudedhia@hotmail.com\r\n,9322233469,Confirmed,,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...
4758,9,KCI,Karma Club,131459,Uday V,Gada,rajudedhia@hotmail.com\r\n,9322233469,Confirmed,,...,Currently Working On,HIMALAYAN SKIING & WINTER CARNIVAL 4-8 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,Himalayan Skiing & Winter Carnival Adventure,https://karmagroup.com/?p=82017&post_type=karm...,8,,,,Himalayan Skiing & Winter Carnival Adventure (...


In [11]:
new_df.to_excel(r"C:\Users\fajar.fatoni\Documents\Python\Data\newsletter_zoho.xlsx", index=False)